This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'VNpuCzKZBS1z24CU1tpt'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
response = requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?api_key=" + API_KEY)
afx_x = response.json()

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

# -- Note: afx_x is a phython dictionary that has the key "dataset". Its corresponding value has the following keys
print(list(afx_x['dataset'].keys()))


['id', 'dataset_code', 'database_code', 'name', 'description', 'refreshed_at', 'newest_available_date', 'oldest_available_date', 'column_names', 'frequency', 'type', 'premium', 'limit', 'transform', 'column_index', 'start_date', 'end_date', 'data', 'collapse', 'order', 'database_id']


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
#1

#Filter out 2017 stock information

afx_x_dataset = afx_x['dataset']
afx_x_data = afx_x_dataset['data']

column_names = afx_x_dataset['column_names']

afx_x_2017_rows = []
afx_x_2017 = {}

for stock_info_at_date in afx_x_data:
    date = stock_info_at_date[0]
    year = date[0:4]
    if year == '2017': afx_x_2017_rows.append(stock_info_at_date)
        
#Structure the data as a dictionary where the keys are the column names and values are from the row items
for column_name in column_names:
    afx_x_2017[column_name] = []
    
for row in afx_x_2017_rows:
    for column_name, item in zip(column_names, row):
        afx_x_2017[column_name].append(item)
        
#Check if afx_x_2017 is consistent with afx_x_2017_rows

for i, column_name in enumerate(column_names):
    column_values_dict = afx_x_2017[column_name]
    column_values_list = [row[i] for row in afx_x_2017_rows]
    if column_values_dict != column_values_list:
        print("Error: column values do not match")
        
#2 is already done

In [6]:
#3
opening_stock_prices = [price for price in afx_x_2017['Open'] if price is not None]
print("Highest opening stock price: ", max(opening_stock_prices))
print("Lowest opening stock price: ", min(opening_stock_prices))

Highest opening stock price:  53.11
Lowest opening stock price:  34.0


In [7]:
#4
high_low_difference = [
    high - low 
    for high, low in zip(afx_x_2017['High'], afx_x_2017['Low'])
]

largest_high_low_difference = max(high_low_difference)

print("Largest change in any one day (based on high and low prices): ", largest_high_low_difference)

Largest change in any one day (based on high and low prices):  2.8100000000000023


In [8]:
#5

closing_prices = afx_x_2017['Close']
max_closing_price = max(closing_prices)
min_closing_price = min(closing_prices)
largest_closing_price_difference_1 = max_closing_price - min_closing_price

print("Largest change between any two days (based on closing price): ", largest_closing_price_difference_1)

largest_closing_price_difference_2 = max(
    [
        price_2 - price_1 
        for price_1, price_2 in zip(closing_prices, closing_prices[1:])
    ]
)
print("Largest change between two consecutive days (based on closing price): ", largest_closing_price_difference_2)

Largest change between any two days (based on closing price):  19.03
Largest change between two consecutive days (based on closing price):  2.559999999999995


In [9]:
#6
trading_volumes = afx_x_2017['Traded Volume']
average_trading_volume = sum(trading_volumes) / len(trading_volumes)

print("Average trading volume: ", average_trading_volume)

Average trading volume:  89124.33725490196


In [10]:
#7
def median(lst):
    sorted_list = sorted(lst)
    
    if len(lst) % 2 != 0:
        mid = (len(lst) - 1) // 2
        
        return sorted_list[mid]
    else:
        mid_l = (len(lst) - 1) // 2
        mid_u = mid_l + 1
        
        return (sorted_list[mid_l] + sorted_list[mid_u]) / 2
    
median_trading_volumes = median(trading_volumes)
print("Median trading volume: ", median_trading_volumes)

Median trading volume:  76286.0
